In [10]:
import pandas as pd
import openpyxl
import os
import glob
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from category_encoders import HashingEncoder

In [11]:
# load data

df_centralised_sentinel_imputed = pd.read_parquet('df_centralised_sentinel_imputed_cleaned.parquet')
df_centralised_cleaned = pd.read_parquet('df_centralised_cleaned.parquet')
df_centralised_sentinel_cleaned = pd.read_parquet('df_centralised_sentinel_cleaned2.parquet')

df_centralised_sentinel_imputed.columns

Index(['Year', 'Name', 'State', 'RegionName', 'SiteDescription', 'TrialCode',
       'SowingDate', 'HarvestDate', 'Abandoned', 'Range', 'Row', 'CultivarID',
       'Harvest Length', 'Harvest Width', 'Kg/Plot', 'T.Max_avg', 'Smx_avg',
       'T.Min_avg', 'Smn_avg', 'Rain_avg', 'Srn_avg', 'Evap_avg', 'Sev_avg',
       'Radn_avg', 'Ssl_avg', 'VP_avg', 'Svp_avg', 'RHmaxT_avg', 'RHminT_avg',
       'B2', 'B3', 'B4', 'B5', 'B8', 'B8A', 'B11', 'ndvi', 'evi', 'evi2',
       'gndvi', 'avi', 'savi', 'osavi', 'vari', 'ndmi', 'msi', 'sipi', 'ndre1',
       'ndre2', 'dvi', 'cirededge', 'ccci', 'Soil_TrialCode', 'Soil_Year',
       'Soil_Released', 'Soil_SoilTestID', 'Soil_MinDepth', 'Soil_MaxDepth',
       'Soil_Aluminium', 'Soil_Ammonium Nitrogen', 'Soil_Boron',
       'Soil_Calcium', 'Soil_Chloride', 'Soil_Conductivity (EC)',
       'Soil_Copper', 'Soil_Exchangeable Al', 'Soil_Exchangeable Ca',
       'Soil_Exchangeable K', 'Soil_Exchangeable Mg', 'Soil_Exchangeable Mn',
       'Soil_Exchangeable

In [ ]:
X_cols = ['TrialCode', 'Range', 'Row', 'CultivarID',
       'Harvest Length', 'Harvest Width', 'T.Max_avg', 'Smx_avg',
       'T.Min_avg', 'Smn_avg', 'Rain_avg', 'Srn_avg', 'Evap_avg', 'Sev_avg',
       'Radn_avg', 'Ssl_avg', 'VP_avg', 'Svp_avg', 'RHmaxT_avg', 'RHminT_avg',
       'B2', 'B3', 'B4', 'B5', 'B8', 'B8A', 'B11', 'ndvi', 'evi', 'evi2',
       'gndvi', 'avi', 'savi', 'osavi', 'vari', 'ndmi', 'msi', 'sipi', 'ndre1',
       'ndre2', 'dvi', 'cirededge', 'ccci', 'Soil_Year',
       'Soil_Released', 'Soil_SoilTestID', 'Soil_MinDepth', 'Soil_MaxDepth',
       'Soil_Aluminium', 'Soil_Ammonium Nitrogen', 'Soil_Boron',
       'Soil_Calcium', 'Soil_Chloride', 'Soil_Conductivity (EC)',
       'Soil_Copper', 'Soil_Exchangeable Al', 'Soil_Exchangeable Ca',
       'Soil_Exchangeable K', 'Soil_Exchangeable Mg', 'Soil_Exchangeable Mn',
       'Soil_Exchangeable Na', 'Soil_Gravel', 'Soil_Magnesium',
       'Soil_Manganese', 'Soil_Nitrate Nitrogen', 'Soil_Organic Carbon',
       'Soil_pH (CaCl2)', 'Soil_pH (water)', 'Soil_Phosphorous',
       'Soil_Potassium (K)', 'Soil_Reactive Iron', 'Soil_Sulphur',
       'Soil_Texture', 'Soil_Zinc']

X = df_centralised_sentinel_imputed[X_cols]
y = df_centralised_sentinel_imputed['Kg/Plot']

# show whole row
pd.set_option('display.max_columns', None)

encoder = HashingEncoder(cols=['CultivarID', 'TrialCode'], n_components=8)
X = encoder.fit_transform(X, y)


,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,Range,Row,Harvest Length,Harvest Width,T.Max_avg,Smx_avg,T.Min_avg,Smn_avg,Rain_avg,Srn_avg,Evap_avg,Sev_avg,Radn_avg,Ssl_avg,VP_avg,Svp_avg,RHmaxT_avg,RHminT_avg,B2,B3,B4,B5,B8,B8A,B11,ndvi,evi,evi2,gndvi,avi,savi,osavi,vari,ndmi,msi,sipi,ndre1,ndre2,dvi,cirededge,ccci,Soil_Year,Soil_Released,Soil_SoilTestID,Soil_MinDepth,Soil_MaxDepth,Soil_Aluminium,Soil_Ammonium Nitrogen,Soil_Boron,Soil_Calcium,Soil_Chloride,Soil_Conductivity (EC),Soil_Copper,Soil_Exchangeable Al,Soil_Exchangeable Ca,Soil_Exchangeable K,Soil_Exchangeable Mg,Soil_Exchangeable Mn,Soil_Exchangeable Na,Soil_Gravel,Soil_Magnesium,Soil_Manganese,Soil_Nitrate Nitrogen,Soil_Organic Carbon,Soil_pH (CaCl2),Soil_pH (water),Soil_Phosphorous,Soil_Potassium (K),Soil_Reactive Iron,Soil_Sulphur,Soil_Texture,Soil_Zinc
0,0,1,0,0,1,0,0,0,1,1,8.4,2.1,23.106316,25.0,8.787368,25.0,1.433684,25.0,4.128421,25.0,16.717895,25.0,11.624737,25.0,42.42,91.931053,261.284748,396.983095,563.542435,682.414779,990.644388,1021.493841,1297.041864,0.292152,0.250429,0.241101,0.436984,0.05909,0.197305,0.176238,-0.202763,-0.121667,1.378164,2.118012,0.1229,0.167451,426.130463,0.587503,0.295162,2015.0,1.0,13196.5,5.0,35.0,8.0,2.6,0.771,2916.666,9.0,0.13,1.028,0.1,24.0,0.26,9.5,0.7104,0.91,3.5,695.16,30.56,10.55,0.676,6.75,7.7,23.0,200.0,25.58,4.58,5.0,0.718
1,1,1,0,0,0,0,0,0,1,2,8.4,2.1,23.106316,25.0,8.787368,25.0,1.433684,25.0,4.128421,25.0,16.717895,25.0,11.624737,25.0,42.42,91.931053,261.284748,396.983095,563.542435,682.414779,990.644388,1021.493841,1297.041864,0.292152,0.250429,0.241101,0.436984,0.05909,0.197305,0.176238,-0.202763,-0.121667,1.378164,2.118012,0.1229,0.167451,426.130463,0.587503,0.295162,2015.0,1.0,13196.5,5.0,35.0,8.0,2.6,0.771,2916.666,9.0,0.13,1.028,0.1,24.0,0.26,9.5,0.7104,0.91,3.5,695.16,30.56,10.55,0.676,6.75,7.7,23.0,200.0,25.58,4.58,5.0,0.718
2,0,2,0,0,0,0,0,0,1,3,8.3,2.1,23.106316,25.0,8.787368,25.0,1.433684,25.0,4.128421,25.0,16.717895,25.0,11.624737,25.0,42.42,91.931053,261.284748,396.983095,563.542435,682.414779,990.644388,1021.493841,1297.041864,0.292152,0.250429,0.241101,0.436984,0.05909,0.197305,0.176238,-0.202763,-0.121667,1.378164,2.118012,0.1229,0.167451,426.130463,0.587503,0.295162,2015.0,1.0,13196.5,5.0,35.0,8.0,2.6,0.771,2916.666,9.0,0.13,1.028,0.1,24.0,0.26,9.5,0.7104,0.91,3.5,695.16,30.56,10.55,0.676,6.75,7.7,23.0,200.0,25.58,4.58,5.0,0.718
3,0,1,0,0,0,0,0,1,1,4,8.4,2.1,23.106316,25.0,8.787368,25.0,1.433684,25.0,4.128421,25.0,16.717895,25.0,11.624737,25.0,42.42,91.931053,261.284748,396.983095,563.542435,682.414779,990.644388,1021.493841,1297.041864,0.292152,0.250429,0.241101,0.436984,0.05909,0.197305,0.176238,-0.202763,-0.121667,1.378164,2.118012,0.1229,0.167451,426.130463,0.587503,0.295162,2015.0,1.0,13196.5,5.0,35.0,8.0,2.6,0.771,2916.666,9.0,0.13,1.028,0.1,24.0,0.26,9.5,0.7104,0.91,3.5,695.16,30.56,10.55,0.676,6.75,7.7,23.0,200.0,25.58,4.58,5.0,0.718
4,0,1,0,1,0,0,0,0,1,5,8.5,2.1,23.106316,25.0,8.787368,25.0,1.433684,25.0,4.128421,25.0,16.717895,25.0,11.624737,25.0,42.42,91.931053,261.284748,396.983095,563.542435,682.414779,990.644388,1021.493841,1297.041864,0.292152,0.250429,0.241101,0.436984,0.05909,0.197305,0.176238,-0.202763,-0.121667,1.378164,2.118012,0.1229,0.167451,426.130463,0.587503,0.295162,2015.0,1.0,13196.5,5.0,35.0,8.0,2.6,0.771,2916.666,9.0,0.13,1.028,0.1,24.0,0.26,9.5,0.7104,0.91,3.5,695.16,30.56,10.55,0.676,6.75,7.7,23.0,200.0,25.58,4.58,5.0,0.718


## Random Forest

### Train Test split

In [24]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [ ]:
# random forest regression

#gridsearch for hyperparameter tuning
param_grid = {
    'n_estimators': [400],
    'max_depth': [None],
    'min_samples_split': [2],
    'min_samples_leaf': [4],
    'max_features': ['sqrt', 'log2'],
    'bootstrap': [True, False],
    'criterion': ['squared_error', 'absolute_error']
}


# Create a Random Forest Regressor model
rf_model = RandomForestRegressor(random_state=42)
# Create a GridSearchCV object
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
# Fit the grid search to the data
grid_search.fit(X_train, y_train)
# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_
print(f'Best Parameters: {best_params}')
print(f'Best Score: {best_score}')
#Best Parameters: {'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 400}
#Best Score: -0.4679382443312489
#Best Parameters: {'bootstrap': False, 'criterion': 'squared_error', 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 400}
#Best Score: -0.5031727162647919

Best Parameters: {'bootstrap': False, 'criterion': 'squared_error', 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 400}
Best Score: -0.5031727162647919


In [ ]:


# Create a Random Forest Regressor model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
# Fit the model to the training data
rf_model.fit(X_train, y_train)
# Make predictions on the test data
y_pred = rf_model.predict(X_test)
# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')